# Configuration of Spark Session to Spark Cluster

In [1]:
from pyspark.sql import SparkSession
from operator import add
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("tim-wywiol-intro")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/21 11:15:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/21 11:15:51 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


# Loading Data in SPARK RDD from HDFS

In [2]:
rdd_lines_en = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.de-en.en")
rdd_lines_de = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.de-en.de")


# A1 

In [3]:
## A1 
## Count lines
print(rdd_lines_en.first())
print(rdd_lines_en.getNumPartitions())
print(rdd_lines_en.count())
print(rdd_lines_de.first())
print(rdd_lines_de.getNumPartitions())
print(rdd_lines_de.count())

Resumption of the session
3


1920209
Wiederaufnahme der Sitzungsperiode
3


1920209


# A2

In [4]:
## A2
#LowerCase RDD Text
def TokLow(line):
    new_line = line.lower()
    return new_line.split()
rdd_lines_en_low = rdd_lines_en.map(lambda x: TokLow(x))
rdd_lines_de_low = rdd_lines_de.map(lambda x: TokLow(x))

In [5]:
print(rdd_lines_en_low.take(2))
print(rdd_lines_en_low.count())
print(rdd_lines_de_low.take(2))
print(rdd_lines_de_low.count())

[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.']]


1920209
[['wiederaufnahme', 'der', 'sitzungsperiode'], ['ich', 'erkläre', 'die', 'am', 'freitag,', 'dem', '17.', 'dezember', 'unterbrochene', 'sitzungsperiode', 'des', 'europäischen', 'parlaments', 'für', 'wiederaufgenommen,', 'wünsche', 'ihnen', 'nochmals', 'alles', 'gute', 'zum', 'jahreswechsel', 'und', 'hoffe,', 'daß', 'sie', 'schöne', 'ferien', 'hatten.']]


1920209


# A3

In [6]:
## A3
# Count 10 most frequent words
# Map
all_words_en = rdd_lines_en_low\
                                .flatMap(lambda w: w)\
                                .map(lambda l: (l,1))
print(all_words_en.take(20))


[('resumption', 1), ('of', 1), ('the', 1), ('session', 1), ('i', 1), ('declare', 1), ('resumed', 1), ('the', 1), ('session', 1), ('of', 1), ('the', 1), ('european', 1), ('parliament', 1), ('adjourned', 1), ('on', 1), ('friday', 1), ('17', 1), ('december', 1), ('1999,', 1), ('and', 1)]


In [7]:
all_words_counts_en = all_words_en.reduceByKey(add)
print(all_words_counts_en.takeOrdered(10, key=lambda x: -x[1]))

[('the', 3663317), ('of', 1737096), ('to', 1611846), ('and', 1345289), ('in', 1134117), ('that', 835926), ('a', 810820), ('is', 792599), ('for', 557377), ('we', 551272)]


In [8]:
## A3 Deutsch
all_words_de = rdd_lines_de_low\
                                .flatMap(lambda w: w)\
                                .map(lambda l: (l,1))
all_words_counts_de = all_words_de.reduceByKey(add)
print(all_words_counts_de.takeOrdered(10, key=lambda x: -x[1]))

[('die', 1980588), ('der', 1710400), ('und', 1338008), ('in', 781398), ('zu', 618887), ('den', 577690), ('wir', 489054), ('für', 478353), ('ich', 469069), ('das', 466167)]


# A4

In [9]:
#######  A4   #################################################################
#1 key line ((1, [w1, w2, w3]), (2, [w1,w2,w3])...
en_1  = rdd_lines_en_low.zipWithIndex()
#[(['resumption', 'of', 'the', 'session'], 0),
en_2 = en_1.map(lambda x: (x[1], x[0]))
print(en_2.take(3))

[(0, ['resumption', 'of', 'the', 'session']), (1, ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.']), (2, ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'])]


In [10]:
de_1  = rdd_lines_de_low.zipWithIndex()
de_2 = de_1.map(lambda x: (x[1], x[0]))

In [11]:
#4.3 Join RDDs
print(en_2.take(2))
print(de_2.take(2))
join_3 = en_2.join(de_2)
print(join_3.take(5))

[(0, ['resumption', 'of', 'the', 'session']), (1, ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'])]
[(0, ['wiederaufnahme', 'der', 'sitzungsperiode']), (1, ['ich', 'erkläre', 'die', 'am', 'freitag,', 'dem', '17.', 'dezember', 'unterbrochene', 'sitzungsperiode', 'des', 'europäischen', 'parlaments', 'für', 'wiederaufgenommen,', 'wünsche', 'ihnen', 'nochmals', 'alles', 'gute', 'zum', 'jahreswechsel', 'und', 'hoffe,', 'daß', 'sie', 'schöne', 'ferien', 'hatten.'])]


[(11310, (['the', 'operation', 'of', 'the', 'community', 'support', 'framework', 'is', 'ruled', 'by', 'a', 'centralising', 'hand', 'and', 'indeed', 'at', 'this', 'time', 'we', 'are', 'experiencing', 'state,', 'government', 'and', 'party', 'delirium', 'as', 'regards', 'the', 'european', "union'", 's', 'funding.'], ['das', 'gemeinschaftliche', 'förderkonzept,', 'so', 'wie', 'es', 'derzeit', 'funktioniert,', 'wird', 'von', 'zentralistischer', 'hand', 'dirigiert,', 'und', 'momentan', 'erleben', 'wir', 'im', 'hinblick', 'auf', 'die', 'beihilfen', 'der', 'europäischen', 'union,', 'daß', 'staat,', 'regierung', 'und', 'partei', 'vollkommen', 'durchdrehen.'])), (17952, (['the', 'very', 'political', 'essence', 'of', 'the', 'reform', 'is', 'mirrored', 'in', 'the', 'objectives', 'of', 'this', 'five-year', 'plan.'], ['in', 'diesen', 'zielsetzungen', 'des', 'fünfjahresprogramms', 'liegen', 'die', 'tieferen', 'politischen', 'gründe', 'für', 'die', 'reform.'])), (19722, (['mr', 'president,', 'as', 'a'

In [12]:
## A4.4 Exclude
# Exclude missing corresponding sentences
en_4 = join_3 \
    .filter(lambda x: len(x[1][0]) != 0) \
    .filter(lambda x: len(x[1][1]) != 0) \
    .filter(lambda x: len(x[1][0]) < 10) \
    .filter(lambda x: len(x[1][1]) == len(x[1][0])) 
         
print(en_4.take(5))

[(107106, (['it', 'must', 'also', 'be', 'broadened.'], ['es', 'muss', 'auch', 'verbreitert', 'werden.'])), (244854, (['this', 'is', 'not', 'a', 'short-term', 'project.'], ['diese', 'unterstützung', 'ist', 'kein', 'kurzfristiges', 'projekt.'])), (500196, (['the', 'olaf', 'report', 'painted', 'a', 'quite', 'different', 'picture.'], ['der', 'olaf-bericht', 'stellte', 'die', 'situation', 'völlig', 'anders', 'dar.'])), (661002, (['.'], ['.'])), (63978, (['then', 'there', 'was', 'the', 'tampere', 'summit.'], ['dann', 'folgte', 'der', 'gipfel', 'von', 'tampere.']))]


In [13]:
## A4.7
en_7 = en_4.flatMap(lambda x: tuple(zip(x[1][0], x[1][1])))
print(en_7.take(4))

[('(applause)', '(beifall)'), ('because,', 'wenn'), ('if', 'nicht,'), ('not,', 'müsste')]


In [14]:
en_8 = en_7 \
    .map(lambda x: (x,1)) \
    .reduceByKey(add)


In [15]:
print(en_8.takeOrdered(20, key=lambda x: -x[1]))

[(('is', 'ist'), 11734), (('the', 'die'), 9815), (('debate', 'aussprache'), 4578), (('we', 'wir'), 4412), (('i', 'ich'), 4391), (('closed.', 'geschlossen.'), 4329), (('the', 'der'), 3085), (('that', 'das'), 2802), (('(applause)', '(beifall)'), 2746), (('.', '.'), 2644), (('and', 'und'), 2490), (('this', 'das'), 1894), (('mr', 'herr'), 1846), (('not', 'nicht'), 1703), (('thank', 'vielen'), 1593), (('a', 'eine'), 1454), (('are', 'sind'), 1449), (('a', 'ein'), 1440), (('by', 'von'), 1434), (('no', 'nr.'), 1407)]


### release the cores for another application!
spark_context.stop()